# Seasonal Regression

The electricity demand series shows daily, weekly, and annual oscillations.  At a short time scale, I aim to capture the first two of these.

The approaches I've seen suggest Fourier Series, linear regression, and Seasonal ARIMA.
(I got quite stuck on how to detrend the series in a global fashion.)
I will focus on building models on the last two weeks of data, with the goal of predicting electricity demand based on temperature, time of day, and day of week.

## Hyndman's Multiple Seasonal Exponential Smoothing

This follows Rob Hyndman's approach towards multi-seasonal exponential smoothing.  (This generalizes the apparently well-known Holt-Winters smoothing).
His analysis includes forecasts of electricity generation, based on utility data (from well over 10 years ago).

I chose to follow this model since initial attempts at ARIMA rely on removing the seasonality, and I had hoped to just follow best practice with existing libraries.  Initial naive methods gave complete crap, and failed to remove the seasonal pattern, or even worse imposed one.  An initial attempt at Fourier filtering on over a year of data also left a 

Hyndman also seems to be a known author within the field of econometric time-series forecasting.  

The original model for a variable $y_t$, with seasonal pattern with period $m$ is
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +s_{t-m} +\epsilon_t\\
  l_t &= l_{t-1} + \alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
  s_{t} = s_{t-m} + \gamma \epsilon_t
\end{align}
where $l_t$, b_t,s_t$ are the level, trend and seasonal patterns respectively.
The noise is Gaussian and obeys
$E[\epsilon_t]=0, E[\epsilon_t\epsilon_s]=\delta_{ts}\sigma^2$, and $\alpha,\beta,\gamma$ are constants between zero and one.  (He notes that $m+2$ estimates must be made for the initial values of the level, trend and seasonal pattern).

Hyndman's model allows multiple seasons, and allows the sub-seasonal terms to be updated more quickly than once per large season.  In utility data, the short season is the daily oscillation, while the longer season comes from the weekly oscillation induced by the work week.  For hourly data, the daily cycle has length $m_1=24$, with the weekly cycle taking $m_2=168$.  The ratio between them is $k=m_2/m_1=7.$  The number of seasonal patterns is $r\le k$.  

(I'm going to change Hyndman's notation to use $\mathbf{I}$ to denote indicator/step functions).
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +\sum_{i=1}^r \mathbf{I}_{t,i}s_{i,t-m_1} +\epsilon_t\\
  s_{i,t} = s_{i,t-m_1} + \sum_{j=1}^r\left(\gamma_{ij}\mathbf{I}_{t,j}\right) \epsilon_t  (i=1,2,\ldots,r)
  l_t &= l_{t-1} + b_{t-1}+\alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
\end{align}
Here the indicator functions $\mathbf{I}_{t,i}$ are unity if $t$ is in the seasonal pattern $i$, and zero otherwise.  For utility data, this will probably be weekday and holiday/weekend.  Here $\gamma_{ij}$ denotes how much one seasonal pattern is updated based on another---Hyndman proposes a number of restrictions on these parameters.

I will extend this to include an external variables for the deviation above a given temperature, so that $y_t\rightarrow y_t+\tau_p\Theta(T_t-T_p)+\tau_{n}\Theta(T_n-T_t)$.

He suggests using the first four weeks of data to estimate the parameters, by minimizing the squared error of the one-step ahead forecast.  Apparently maximum likelihood estimation was not recommended (10 years ago).

So how to fit the parameters?  A really simple approach would be gradient descent?  Intuitively, the level is the average value, the bias is the average gradient.  The seasonality is the average seasonal pattern.  (This is the dumb STL decomposition used earlier?)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from get_weather_data import convert_isd_to_df, convert_state_isd
from EBA_util import remove_na, avg_extremes

In [2]:
air_df = pd.read_csv('data/air_code_df.gz')

#Just get the weather station data for cities in Oregon.
df_weather=convert_state_isd(air_df,'OR')
#Select temperature for Portland, OR
msk1=np.array(df_weather['city']=='Portland')
msk2=np.array(df_weather['state']=='OR')

df_pdx_weath=df_weather.loc[msk1&msk2]

#get electricity data for Portland General Electric
df_eba=pd.read_csv('data/EBA_time.gz',index_col=0,parse_dates=True)
msk=df_eba.columns.str.contains('Portland')
df_pdx=df_eba.loc[:,msk]



done with Mahlon Sweet Field


done with Salem Municipal Airport/McNary Field


done with Portland International Airport


In [3]:
dem=df_pdx.iloc[:,0]
#Make a combined Portland Dataframe for demand vs weather.
df_joint=pd.DataFrame(dem)
df_joint=df_joint.join(df_pdx_weath)
temp=df_joint['Temp']
df_joint['TempShift']=150+abs(temp-150)
df_joint=df_joint.rename(columns={df_joint.columns[0]:'Demand'})


In [4]:
#clean up data, remove NA
dem = remove_na(dem)
dem = avg_extremes(dem)

temp = avg_extremes(remove_na(temp))

Number of extreme values 0. Number of zero values 148


Number of NA values 56


Number of extreme values 1. Number of zero values 3


Number of NA values 156


In [5]:
t=np.arange(168)
dem_sub=dem[0:4*24*7]


In [6]:
def fit_init_params(y):
    """fit_init_params(y)
    Fits initial parameters for Hyndman's multi-seasonal model to
    hourly electricity data.
    (My guess on how to do this, similar to naive STL method used in 
    statstools.timeseries)

    Finds level, bias and seasonal patterns based on first 4 weeks of data.  
    """
    m1 = 24
    k = 7
    ysub = y[0:4*24*7]
    yval = ysub.values
    #average value
    l = np.mean(yval)
    #average shift
    b = np.mean(np.diff(yval))
    #remove mean pattern, subtractin off level, and linear trend.    
    ysub = ysub-l-b*np.arange(4*24*7)

    #mean seasonal pattern.
    #second seasonal pattern is for weekends, with days
    #Saturday/Sunday have dayofweek equal to 5 and 6.
    #make a mask to select out weekends.
    s2 = ysub.index.dayofweek >=5
    #select out weekends, and regular days. 
    y_end = ysub[s2]
    y_week=ysub[~s2]
    n1 = int(len(y_week)/24)
    n2 = int(len(y_end)/24)
    s = np.zeros((2,24))
    print(n1,n2)
    for n in range(n1):
        s[0,:] = s[0,:]+y_week[n*24:(n+1)*24]/n1
    for n in range(n2):
        s[1,:] = s[1,:]+y_end[n*24:(n+1)*24]/n2

    return l, b, s

def predict_stl(l,b,s,timeIndex):
    """predict_stl(l,b,s,times)

    """
    trend=l+b*np.arange(len(timeIndex))
    #find weekend/weekedays.  
    msk=timeIndex.dayofweek>=5
    #
    n1 = int(sum(~msk)/24)    
    n2 = int(sum(msk)/24)
    #Use fact that first sub-season is weekdays in first row.
    #Use integer conversion of true/false to 0/1.
    #Then use fact that seasonal patterns are 24 hours long.
    pred=trend+s[msk.astype(int),timeIndex.hour.values]
    return pred

def update_params(param_vec,grad_vec,actual,predicted):
    """update_params
    Change parameters based on gradients, and difference
    between predicted and actual values.
    
    grad_vec - list of functions to evaluate at parameters

    Work in Progress
    """
    param_new=param_vec.copy()
    nparam=len(param_vec)
    for i in range(nparam):
        param_new[i] += grad_vec[i](param_vec)*(actual-predicted)
    return param_new


Let's just try to build a linear model for the temperature on top of this.
I'll assume that heating/cooling might have different coefficients, so the temperature component of the model at time $t$ is
\begin{equation}
D_t =  a_0+ a_+[T_t-T_{+}]_{+} + a_-[T_{-}-T_t]_{+},
\end{equation}
where $[f]_+=f$ if $f>0$, and is zero otherwise.  If we optimize the mean square
error, then the components can be found by solving for the values that minimize the derivatives.
If criteria is mean square error, then can solve directly for parameters
$J = sum_t[\hat{D}_t-D_t)^2$, where $\hat{D}_t$ is the true value.
We must solve $\partial J/\partial \alpha = 0 \rightarrow \sum_t \frac{\partial D_t}{\partial\alpha}(\hat{D}_t-D_t) $

Those conditions are
\begin{align}
    \sum_t(\hat{D}_t-D_t)=0  \qquad (a_0)\\
    \sum_{t\in T_\pm}[T_t-T_\pm]_\pm(\hat{D}_t-D_t)=0  \qquad (a_\pm)\\
    \sum_{t\in T_\pm}(\pm a_{\pm})(\hat{D}_t-D_t)=0  \qquad (T_\pm)\\
\end{align}


SyntaxError: unexpected EOF while parsing (<ipython-input-80-1a9f42a86693>, line 22)

(First OOP class - absolutely ridiculous)  Will just use a pandas Dataframe - want a named set of numbers, with defined operations.  Already here.  

In [85]:
D=dem
T=temp
Dr = np.max(D)-np.min(D)
Tr = np.max(T)-np.min(T)

pnames=['a0','ap','an','Tp','Tn']
pval=[np.mean(D),0.5*Dr/Tr,0.5*Dr/Tr,200,100]
param=param_vec(dict(zip(pnames,pval)))
pval2=[1000,100,100,10,20]
param1=param_vec(dict(zip(pnames,pval2)))
p2=param-param1
p3=param-param


0.0

In [104]:
#Experimenting with OOP for making "vector" of parameters with named labels.
#Feels daft - if there's a less stupid way, I'll try to fix this.
#Initial attempts at getting smarter initialization (with arguments to make a dict just returned empty.
class param_vec(dict):
    """Class for model parameters.
    Stores parameters in dict, with arithmatic operations.
    """
    #Define elementwise subtraction/addition
    def __add__(self,x):
        y = self.copy()
        if isinstance(x,(int,float)):
            for i,v in self.items():
                y[i]=self[i]+x
        else:
            for i,v in self.items():
                y[i]=self[i]+x[i]
        return param_vec(y)


    #Define elementwise subtraction/addition
    def __radd__(self,x):
        return param_vec.__add__(self,x)
    
    #Define elementwise subtraction.
    def __sub__(self,x):
        y=self.copy()
        if isinstance(x,(int,float)):
            for i,v in self.items():
                y[i]=v-x
        else:        
            for i,v in self.items():
                y[i]=v-x[i]
        return param_vec(y)
                
    #Define elementwise subtraction/addition
    def __rsub__(self,x):
        return param_vec.__sub__(self,x)

                
    #Define elementwise subtraction.
    def __truediv__(self,x):
        y=self.copy()
        if isinstance(x,(int,float)):
            for i,v in self.items():
                y[i]=v/x
        else:        
            for i,v in self.items():
                y[i]=v/x[i]
        return param_vec(y)
                
    #Define elementwise subtraction.
    def __mul__(self,x):
        y=self.copy()
        if isinstance(x,(int,float)):
            for i,v in self.items():
                y[i]=v*x
        else:        
            for i,v in self.items():
                y[i]=v*x[i]
        return param_vec(y)
                
    #Define elementwise subtraction.
    def __rmul__(self,x):
        return param_vec.__mul__(self,x)

#might extend to contain all of the modelling stuff?          

In [107]:
def temp_model(T,param):    
    m1 = T>param['Tp']
    m2 = T<param['Tn']
    y=np.zeros(T.shape)
    y[m1] = (T[m1]-param['Tp'])*param['ap']
    y[m2] = (T[m2]-param['Tn'])*param['an']
    y=y+param['a0']
    return y

def temp_model_grad(Dhat,D,T,param):
    """temp_model_grad
    Compute gradients of model w.r.t. parameters.
    Assumes loss-function is mean-square.
    """
    m1 = T>param['Tp']
    m2 = T<param['Tn']
    Derr=Dhat-D
    dparam=param_vec(dict( zip(param.keys(),np.zeros(len(param)) )))
    dparam['a0'] = np.sum(Derr)
    y=np.zeros(T.shape)

    #\sum_{t\in T_\pm}[T_t-T_\pm]_\pm(\hat{D}_t-D_t)=0  \qquad (a_\pm)\\
    dparam['ap'] = np.sum( (T[m1]-param['Tp'])*(Derr[m1]))
    dparam['an'] = np.sum( (param['Tn']-T[m2])*(Derr[m2]))

    #\sum_{t\in T_\pm}(\pm a_{\pm})(\hat{D}_t-D_t)=0  \qquad (T_\pm)\\
    dparam['Tp'] = param['ap']*np.sum(Derr[m1])
    dparam['Tn'] = -param['an']*np.sum(Derr[m2])    
    return dparam

def param_fit(Dhat,T,alpha=0.1,rtol=1E-3,nmax=10):
    """Try to fit linear threshold model of demand to temperature.

    D - demand data
    T - temperature data

    Fits model of form:
    D ~ a_0+ a_p[T-T_p]_+ + a_n[T_n-T]_+,
    where [f]_+ =f for f>0, and 0 otherwise.

    Just use simple gradient descent to fit the model.
    """
    #make parameter estimates
    Dr = np.max(Dhat)-np.min(Dhat)
    Tr = np.max(T)-np.min(T)
    param_names=['a0','ap','an','Tp','Tn']
    param_vals=[np.mean(Dhat), 0.5*Dr/Tr, 0.5*Dr/Tr, 200, 100]
    param=param_vec(dict(zip(param_names,param_vals)))
    Dpred = temp_model(T,param)                                    
    mean_err=10*rtol
    Ni=0
    for i in range(nmax):
        dparam=temp_model_grad(Dpred,Dhat,T,param)
        param=param-alpha*dparam
        Dpred2=temp_model(T,param)
        mean_err_change=abs(1-np.mean(Dpred2/Dpred))
        Dpred=Dpred2
        Ni+=1
        if (mean_err_change<rtol):
           print("Hit tolerance {} at iter {}".format(mean_err_change,Ni))
           return param,Dpred
        #if(Ni%100==0):
        print("Mean Error {} at iter {}".format(
        mean_err_change,Ni))
    print("Failed to hit tolerance after {} iter".format(iter))
    return param, Dpred

In [108]:
dem_sub   = dem[0:10]
temp_sub = temp[0:10]
param,Dpred=param_fit(dem_sub,temp_sub)

<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 89
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 90
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 91
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 92
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 93
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 94
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 95
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 96
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 97
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 98
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 99
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at 

Mean Error nan at iter 76
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 77
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 78
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 79
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 80
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 81
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 82
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 83
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 84
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 85
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 86
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 87
<class '__main__.param_vec'> <class '__main__.para

Mean Error nan at iter 63
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 64
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 65
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 66
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 67
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 68
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 69
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 70
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 71
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 72
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 73
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 74
<class '__main__.param_vec'> <class '__main__.para

Mean Error nan at iter 50
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 51
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 52
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 53
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 54
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 55
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 56
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 57
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 58
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 59
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 60
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 61
<class '__main__.param_vec'> <class '__main__.para

<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 38
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 39
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 40
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 41
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 42
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 43
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 44
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 45
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 46
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 47
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 48
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at 

<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 27
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 28
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 29
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 30
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 31
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 32
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 33
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 34
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 35
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 36
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 37


Mean Error nan at iter 14
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 15
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 16
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 17
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 18
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 19
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 20
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 21
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 22
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 23
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 24
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 25
<class '__main__.param_vec'> <class '__main__.para

/home/jonathan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in true_divide
/home/jonathan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error 6944.781692655522 at iter 1
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error 583404017096758.8 at iter 2
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error 1.9788874138201533e+44 at iter 3
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error 7.749314002861557e+132 at iter 4
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error inf at iter 5
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 6
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 7
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 8
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 9
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 10
<class '__main__.param_vec'> <class '__main__.param_vec'>
Mean Error nan at iter 11
<class '__main__.pa

In [13]:
l,b,s=fit_init_params(dem)

pred=predict_stl(l,b,s,dem_sub.index)

20 8


In [14]:
plt.plot(dem_sub.values,'b',temp[0:4*168].values,'r',pred,'k')
plt.show()

In [76]:
msk=dem_sub.index.dayofweek>=5

s[msk.astype(int),dem_sub.index.hour.values]

array([ 958.51319104,  996.49083635,  961.16848165,  859.94612696,
        739.37377227,  658.95141757,  432.02906288,  146.20670818,
        -83.86564651, -228.03800121, -318.6103559 , -358.7827106 ,
       -336.85506529, -229.27741998,  -52.64977468,  139.67787063,
        269.55551593,  380.48316124,  483.26080654,  580.63845185,
        669.91609715,  760.99374246,  839.62138777,  916.24903307,
        958.51319104,  996.49083635,  961.16848165,  859.94612696,
        739.37377227,  658.95141757,  432.02906288,  146.20670818,
        -83.86564651, -228.03800121, -318.6103559 , -358.7827106 ,
       -336.85506529, -229.27741998,  -52.64977468,  139.67787063,
        269.55551593,  380.48316124,  483.26080654,  580.63845185,
        669.91609715,  760.99374246,  839.62138777,  916.24903307,
        958.51319104,  996.49083635,  961.16848165,  859.94612696,
        739.37377227,  658.95141757,  432.02906288,  146.20670818,
        -83.86564651, -228.03800121, -318.6103559 , -358.78271

1

Rambling Time!

From a Kalman filter perspective, I think that sometimes the error/innovation terms can be written as $\epsilon_t = y_t-\hat{y}_t$, where $y_t$ is the actual value, and $\hat{y}_t$ is the output of the model with no noise.  The innovation process, then gives a rule for updating (the set of parameters $\alpha,\beta,\Gamma, l, b, s_{i,t}$) how to change in the 

In [35]:
%pdb

Automatic pdb calling has been turned OFF
